In [1]:
import sys
import dotenv
dotenv.load_dotenv('../../.env')

sys.path.insert(0, '../')
sys.path.insert(0, '../app')

from app.processing.audio.SIWhisperModel import SIWhisperModel
from app.processing.VideoProcessor import VideoProcessor

whisper = SIWhisperModel('medium', 'whisper-medium')



/home/erwan/anaconda3/envs/scienceinfuse/lib/python3.9/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
[nltk_data] Downloading package omw-1.4 to /home/erwan/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/erwan/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


## process youtube

In [ ]:
from app.S3Storage import S3Storage


s3 = S3Storage()
VideoProcessor(s3=s3, whisper=whisper, youtube_url="https://www.youtube.com/watch?v=abQFa-Nu5V4")

## process s3 object

In [2]:
from app.S3Storage import S3Storage


s3 = S3Storage()
video = VideoProcessor(s3=s3, whisper=whisper, s3_object_name="youtube/5df9290c-4ba5-4643-a3c2-17ab040b35d3.mp4")
document, chunks = video.extract_document()

File youtube/5df9290c-4ba5-4643-a3c2-17ab040b35d3.mp4 successfully downloaded to /home/erwan/Desktop/clients/ScienceInfuse/server/notebooks/documents/temp_videos/8919cbfe-9943-4f97-9702-4b96335af80b.mp4
AUDIO CODEC aac


whisper-medium: 100%|██████████| 172.617/172.617 [00:09<00:00, 17.48s/s]           


time_taken 10.226472616195679
segment.word_segments <class 'app.processing.audio.SIWhisperModel.TranscriptSegment'> text=' ...' start=5.620000000000003 end=6.1 duration=0.4799999999999969
get_video_duration -> Video path: /home/erwan/Desktop/clients/ScienceInfuse/server/notebooks/documents/temp_videos/8919cbfe-9943-4f97-9702-4b96335af80b.mp4


In [7]:
chunks[0].metadata.word_segments

AttributeError: 'list' object has no attribute 'dict'

# tests

In [ ]:
from pytubefix import YouTube
url =     "https://www.youtube.com/watch?v=Ull-Rz1L-J0"
yt = YouTube(url)
video_name = yt.vid_info.get("videoDetails", {}).get("title", "Untitled Video")
# yt.streams.filter(progressive=True, file_extension='mp4').order_by('resolution').desc().first()#.download(output_path=output_path, filename=filename)


In [ ]:
yt.streams

In [ ]:
from googleapiclient.discovery import build

# Set up YouTube API client
api_key = ''
youtube = build('youtube', 'v3', developerKey=api_key)

def get_channel_videos(channel_id):
    playlist_id = f"{channel_id[0]}U{channel_id[2:]}"
    videos = []
    next_page_token = None
    
    while True:
        res = youtube.playlistItems().list(
            part='snippet',
            playlistId=playlist_id,
            maxResults=50,
            pageToken=next_page_token
        ).execute()
        
        for item in res["items"]:
            videos.append({
                "title": item["snippet"]["title"],
                "video_id": item["snippet"]["resourceId"]["videoId"],
                "publish_time": item["snippet"]["publishedAt"]
            })
        
        next_page_token = res.get('nextPageToken')
        
        if next_page_token is None:
            break
            
    return videos

videos = get_channel_videos("UC1udnO-W6gpR9qzleJ5SDKw")
print(len(videos))

In [ ]:
from pytubefix import YouTube
YOUTUBE_PROXY = "socks5://aryakali.freeboxos.fr:33128"
# YOUTUBE_PROXY = "socks5://vm-dev-tools.citepro.cite-sciences.fr:8080/"
proxies = {
    "http": YOUTUBE_PROXY,
    "https": YOUTUBE_PROXY
}
yt = YouTube("https://www.youtube.com/watch?v=YWcoQ8M3cnk",proxies=proxies)

In [ ]:
video_name = yt.vid_info.get("videoDetails", {}).get("title", "Untitled Video")
video_name

In [ ]:
yt.streams.filter(progressive=True, file_extension='mp4').order_by('resolution').desc().first().download(output_path='./', filename='blobremove.mp4')
